In [16]:
import torch
import numpy as np

#### Using tensors
Creating a tensor from a list

In [17]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)
print(x_data)
print(type(x_data))

tensor([[1, 2],
        [3, 4]])
<class 'torch.Tensor'>


Creating a tensor from a numpy array

In [18]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print("Numpy Array:\n", np_array,"\nTorch tensor:\n", x_np)
print(type(x_np))

Numpy Array:
 [[1 2]
 [3 4]] 
Torch tensor:
 tensor([[1, 2],
        [3, 4]], dtype=torch.int32)
<class 'torch.Tensor'>


#### Datasets and Dataloaders

In [22]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [26]:
training_data = datasets.MNIST(
    root="MNISTset",
    train=True,
    download=False,
    transform=ToTensor()
)

RuntimeError: Dataset not found. You can use download=True to download it